# Гены отвечающие за эпигенетику

Геном, аннотация и протеом скачаны напрямую с [Warmbase](https://parasite.wormbase.org/Caenorhabditis_remanei_prjna53967/Info/Index/#:~:text=Assembly%20C_remanei,Source%20WormBase%20Annotation%20Version%20WS290), и сложены в папку `data` 

Протеом находился в готовом виде на сайте WarmBase

P.S остальные файлы в папке дата появится по ходу выполнения ноутбука я просто случайно перезапустил эту ячейку :)

In [5]:
!ls data

remanei_annotations.gff3  remanei_proteins.fa  splitted_genome_6
remanei_genomic.fa	  splitted_genome


Устанавливаем hmmer

In [ ]:
%%bash
sudo apt update
sudo apt install hmmer

In [7]:
!hmmscan -h

# hmmscan :: search sequence(s) against a profile database
# HMMER 3.4 (Aug 2023); http://hmmer.org/
# Copyright (C) 2023 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Usage: hmmscan [-options] <hmmdb> <seqfile>

Basic options:
  -h : show brief help on version and usage

Options controlling output:
  -o <f>           : direct output to file <f>, not stdout
  --tblout <f>     : save parseable table of per-sequence hits to file <f>
  --domtblout <f>  : save parseable table of per-domain hits to file <f>
  --pfamtblout <f> : save table of hits and domains to file, in Pfam format <f>
  --acc            : prefer accessions over names in output
  --noali          : don't output alignments, so output is smaller
  --notextw        : unlimit ASCII text output line width
  --textw <n>      : set max width of ASCII text output lines  [120]  (n>=120)

Options controlling reporting 

Скачиваем базу для hmmer

In [8]:
%%bash
mkdir -p ./pfam_db
wget http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam35.0/Pfam-A.hmm.gz -P ./pfam_db
gzip -d pfam_db/Pfam-A.hmm.gz

--2025-06-18 21:03:12--  http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam35.0/Pfam-A.hmm.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293000230 (279M) [application/x-gzip]
Saving to: ‘./pfam_db/Pfam-A.hmm.gz’

     0K .......... .......... .......... .......... ..........  0%  478K 9m59s
    50K .......... .......... .......... .......... ..........  0% 1006K 7m21s
   100K .......... .......... .......... .......... ..........  0% 13.2M 5m1s
   150K .......... .......... .......... .......... ..........  0% 48.7M 3m47s
   200K .......... .......... .......... .......... ..........  0% 1.03M 3m56s
   250K .......... .......... .......... .......... ..........  0% 34.5M 3m18s
   300K .......... .......... .......... .......... ..........  0% 21.7M 2m52s
   350K .......... .......... .......... .......... ..........  0% 78.4M 2m31s
   400

In [9]:
!hmmpress ./pfam_db/Pfam-A.hmm

Working...    done.
Pressed and indexed 19632 HMMs (19632 names and 19632 accessions).
Models pressed into binary file:   ./pfam_db/Pfam-A.hmm.h3m
SSI index for binary model file:   ./pfam_db/Pfam-A.hmm.h3i
Profiles (MSV part) pressed into:  ./pfam_db/Pfam-A.hmm.h3f
Profiles (remainder) pressed into: ./pfam_db/Pfam-A.hmm.h3p


Поскольку не важно какие семейства будем брать, выберем их случайно из таблицы которая была для их выбора, сохраним название семейства и ассоциированные с ним домены.

In [1]:
import pandas as pd
import re
import json

df = pd.read_csv('selected_table.csv')

def extract_pf_ids(domains_str):
    return re.findall(r'PF\d{5}', domains_str)

df['PF_IDs'] = df['Pfam domains'].fillna('').apply(extract_pf_ids)
gene_pf_map = df[['HGNC approved symbol', 'PF_IDs']].copy()

chosen_families = {}
pf_ids = []

families_count = 15

sample = gene_pf_map.sample(n=families_count * 3, random_state=42)
idx = 0
while len(chosen_families) < families_count and idx < len(sample):
    family = sample.iloc[idx]
    if len(family['PF_IDs']) < 1:
        continue
    
    chosen_families[family['HGNC approved symbol']] = family['PF_IDs']
    pf_ids.extend(list(family['PF_IDs']))
    idx += 1
    
assert idx < len(sample), 'Failed to chose 10 rows'

with open("chosen_families.txt", "w") as f:
    json.dump(chosen_families, f, indent=2)
    
chosen_families['CBLL1'] = ['PF18408']  # для групповой части
pf_ids.append('PF18408')

unique_pf_ids = list(set(pf_ids))

with open("unique_pf_ids.txt", "w") as f:
    for pf_id in unique_pf_ids:
        f.write(pf_id + '\n')
        
        

print('Chosen families:')
for k, v in chosen_families.items():
    print(f'    {k}: ', end='')
    print(*v, sep=', ')

print(f'Unique PF Ids count: {len(unique_pf_ids)}')

Chosen families:
    SETMAR: PF01498, PF05033, PF00856, PF01359
    SETD2: PF00856, PF08236, PF00397
    RBM7: PF00076
    SS18L2: PF05030
    PRDM12: PF13465
    ERCC6: PF00271, PF00176
    MPHOSPH8: PF12796, PF00385
    RAG2: PF03089, PF13341
    GSG2: PF12330, PF00069
    SENP1: PF02902
    MDC1: PF00498
    SP140: PF00439, PF00628, PF01342, PF03172
    WDR5: PF00400
    ZNF532: PF00096
    ZMYM2: PF12012, PF06467
    CBLL1: PF18408
Unique PF Ids count: 28


Поскольку каждый домен еще имеет версионирование в суффиксе (например PF00856 $\to$ PF00856.31) то нам надо дописать их для поиска по базе:

In [11]:
wanted_ids = set(unique_pf_ids)

pattern = re.compile(r'^ACC\s+(PF\d+\.\d+)$')

counter = 0

with open("pfam_db/Pfam-A.hmm") as infile, open("unique_pf_ids_with_versions.txt", 'w') as outfile:
    for line in infile:
        match = pattern.match(line)
        if match:
            full_pf = match.group(1)  # PF00856.33
            prefix = full_pf.split('.')[0]  # PF00856
            if prefix in wanted_ids:
                outfile.write(f"{full_pf}\n")
                counter += 1
                
print(f'Ids found: {counter}')

Ids found: 28


Теперь обьединим все выбранные домены в общий `.hmm` файл, чтобы не делать поиск по всей базе.

In [15]:
%%bash
while read pf; do
  hmmfetch pfam_db/Pfam-A.hmm "$pf" >> pfam_db/epigenetic.hmm
done < unique_pf_ids_with_versions.txt

In [16]:
%%bash
hmmpress pfam_db/epigenetic.hmm

Working...    done.
Pressed and indexed 28 HMMs (28 names and 28 accessions).
Models pressed into binary file:   pfam_db/epigenetic.hmm.h3m
SSI index for binary model file:   pfam_db/epigenetic.hmm.h3i
Profiles (MSV part) pressed into:  pfam_db/epigenetic.hmm.h3f
Profiles (remainder) pressed into: pfam_db/epigenetic.hmm.h3p


Выполним поиск по нашему протеому, флаг `--cut_ga` сразу обрежет совпадения со слабой уверенностью

In [17]:
%%bash
hmmsearch --cpu 8 --cut_ga \
        --domtblout epigenetic.domtblout \
        pfam_db/epigenetic.hmm \
        data/remanei_proteins.fa

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.4 (Aug 2023); http://hmmer.org/
# Copyright (C) 2023 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  pfam_db/epigenetic.hmm
# target sequence database:        data/remanei_proteins.fa
# per-dom hits tabular output:     epigenetic.domtblout
# model-specific thresholding:     GA cutoffs
# number of worker threads:        8
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       Ank_2  [M=83]
Accession:   PF12796.10
Description: Ankyrin repeats (3 copies)
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence Description
    ------- ------ -----    ------- ------ -----   ---- --  -------- -----------
   8.5e-1

Прочитаем наш файл аннотации

In [2]:
gff3_path = 'data/remanei_annotations.gff3'
df_genes = pd.read_csv(
    gff3_path,
    sep='\t',
    comment='#',
    header=None,
    usecols=[0, 2, 3, 4, 6, 8],
    names=['contig', 'feature', 'gene_start', 'gene_end', 'strand', 'attributes']
)

Выделим из него только гены

In [3]:

df_genes = df_genes[df_genes['feature'] == 'gene']

df_genes['sequence_id'] = df_genes['attributes'] \
    .str.extract(r'sequence_name=(CRE\d+)')
df_genes['gene_id'] = df_genes['attributes'] \
    .str.extract(r'Name=(WBGene\d+)')
    
df_genes = df_genes.dropna(subset=['sequence_id'])

df_genes = df_genes[[
    'sequence_id',
    'gene_id',
    'contig',
    'gene_start',
    'gene_end',
    'strand'
]]

Также считаем выход `hammer`, и оставим только нужные нам колонки. На всякий случай еще раз отфильтруем вхождения с высоким `E-value`.

In [4]:
MIN_I_EVALUE = 1e-5
domtbl_path = 'epigenetic.domtblout'

records = []
with open(domtbl_path) as f:
    for line in f:
        if line.startswith('#') or not line.strip():
            continue
        cols = line.split()
        
        i_evalue = float(cols[12])
        if i_evalue > MIN_I_EVALUE:  # отсеиваем домены с плохой надежностью
            continue
        
        records.append({
            'domain_family': cols[4].split('.')[0],
            'sequence_id':   cols[0],
        })
df_domains = pd.DataFrame(records).drop_duplicates()

Теперь обьеденим выход программы с нашей аннотацией, будем матчить их по `sequence_name`, такое поле есть как в протеоме, так и в аннотации.

In [5]:
df_result = pd.merge(
    df_domains,
    df_genes,
    on='sequence_id',
    how='left'
)[[
    'domain_family',
    'gene_id',
    'contig',
    'gene_start',
    'gene_end',
    'strand'
]]

df_result['gene_start'] = df_result['gene_start'].astype('Int64')
df_result['gene_end'] = df_result['gene_end'].astype('Int64')

df_result.to_csv('domain_to_gene_coordinates.csv', index=False)

df_result.head()

,domain_family,gene_id,contig,gene_start,gene_end,strand
0,PF12796,WBGene00075356,Crem_Contig2002,416,4984,-
1,PF12796,WBGene00073146,Crem_Contig65,399237,407403,-
2,PF12796,WBGene00056126,Crem_Contig4,1388780,1397022,+
3,PF12796,WBGene00059223,Crem_Contig18,626325,636798,-
4,PF12796,WBGene00055917,Crem_Contig12,1432840,1436383,-


Теперь обьединим найденные домены по семействам отобранным ранее, поскольку мы хотим понять нашлось то или иное семейство или нет. Будем считать что ген принажлежит семейству по следующему правилу:

Если у семейство всего ли ишь один или два необходимых Pfam домена, то будем требовать полного совпадения с найденными в гене, если больше, то будем требовать чтобы нашлось хотя бы два.

In [6]:
grouped = (
    df_result
      .groupby(['gene_id', 'contig', 'gene_start', 'gene_end', 'strand'])['domain_family']
      .apply(list)
      .reset_index(name='all_domains')
)

results = []

for fam_name, pf_list in chosen_families.items():
    threshold = len(pf_list) if len(pf_list) <= 2 else 2
    
    for _, row in grouped.iterrows():
        present = set(row['all_domains'])
        wanted  = set(pf_list)
        n_matches = len(present & wanted)
        
        if n_matches >= threshold:
            results.append({
                'family': fam_name,
                'gene_id': row['gene_id'],
                'matched_domains': sorted(list(present & wanted)),
                'contig': row['contig'],
                'gene_start': row['gene_start'],
                'gene_end': row['gene_end'],
                'strand': row['strand']
            })

df_matches = pd.DataFrame(results)

df_matches.to_csv('chosen_family_matches.tsv', sep='\t', index=False)

print(f'Found families count: {df_matches.family.nunique()}')

df_matches.head()

Found families count: 10


,family,gene_id,matched_domains,contig,gene_start,gene_end,strand
0,SETMAR,WBGene00054625,"[PF00856, PF05033]",Crem_Contig14,1010208,1011716,+
1,SETMAR,WBGene00062974,"[PF00856, PF05033]",Crem_Contig40,93239,97991,-
2,SETMAR,WBGene00065102,"[PF00856, PF05033]",Crem_Contig64,116933,120275,-
3,SETMAR,WBGene00066923,"[PF00856, PF05033]",Crem_Contig31,861051,863580,+
4,SETMAR,WBGene00066924,"[PF00856, PF05033]",Crem_Contig31,850734,857891,+


Сделаем текст для презентации)

In [ ]:
print(df_matches[['family', 'gene_id', 'matched_domains', 'gene_start', 'gene_end']].drop_duplicates(subset=['family']).to_markdown(index=False))

| family   | gene_id        | matched_domains        |   gene_start |   gene_end |
|:---------|:---------------|:-----------------------|-------------:|-----------:|
| SETMAR   | WBGene00054625 | ['PF00856', 'PF05033'] |      1010208 |    1011716 |
| SETD2    | WBGene00062392 | ['PF00397', 'PF00856'] |       410252 |     415708 |
| RBM7     | WBGene00051716 | ['PF00076']            |      2464513 |    2466527 |
| SS18L2   | WBGene00062405 | ['PF05030']            |       474293 |     475160 |
| ERCC6    | WBGene00051852 | ['PF00176', 'PF00271'] |      3230478 |    3245668 |
| SENP1    | WBGene00052730 | ['PF02902']            |      1818989 |    1821988 |
| MDC1     | WBGene00053500 | ['PF00498']            |      2091575 |    2092962 |
| SP140    | WBGene00066767 | ['PF00439', 'PF00628'] |       176539 |     185416 |
| WDR5     | WBGene00051832 | ['PF00400']            |      3105349 |    3109224 |
| ZNF532   | WBGene00051333 | ['PF00096']            |       625835 |     628021 |
